In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
game_detail = "../dataset/game_detail_v2.json"
user_friend = "../dataset/user_friend_list.json"
user_idx = "../dataset/user_idx.json"
user_owned_games = "../dataset/user_owned_games.json"
user_recently_played_games = "../dataset/user_recently_played_games.json"
user_summary = "../dataset/user_summary.json"

# 1. Convert json to dataframe

In [3]:
game_detail_df = pd.read_json(game_detail, lines = True)
user_frined_df = pd.read_json(user_friend, lines = True)
user_idx_df = pd.read_json(user_idx, lines = True)
user_owned_games_df = pd.read_json(user_owned_games, lines = True)
user_recent_df = pd.read_json(user_recently_played_games, lines = True)
user_summary_df  = pd.read_json(user_summary, lines = True)

In [4]:
print("user_idx columns         :", user_idx_df.columns.values)
print("user_owned_games columns :", user_owned_games_df.columns.values)
print("user_recent_df columns   :", user_recent_df.columns.values)
print("user_summary columns     :", user_summary_df.columns.values)
print("game_detail_df columns   :", game_detail_df.columns.values)

user_idx columns         : ['user_idx' 'user_id']
user_owned_games columns : ['steamid' 'game_count' 'games']
user_recent_df columns   : ['steamid' 'total_count' 'games']
user_summary columns     : ['steamid' 'primaryclanid' 'realname' 'personaname' 'personastate'
 'personastateflags' 'communityvisibilitystate' 'avatarhash'
 'profilestate' 'profileurl' 'loccityid' 'timecreated' 'avatar'
 'locstatecode' 'commentpermission' 'avatarfull' 'avatarmedium'
 'loccountrycode' 'gameextrainfo' 'gameid' 'gameserverip'
 'gameserversteamid' 'lobbysteamid' 'lastlogoff']
game_detail_df columns   : ['steam_appid' 'achievements' 'price_overview' 'demos'
 'detailed_description' 'screenshots' 'genres' 'required_age'
 'about_the_game' 'short_description' 'developers' 'type'
 'supported_languages' 'website' 'publishers' 'pc_requirements'
 'recommendations' 'is_free' 'legal_notice' 'background'
 'controller_support' 'packages' 'categories' 'support_info'
 'content_descriptors' 'name' 'platforms' 'package_gro

# 2. Preprocessing

## (1) convert user id to index

In [76]:
user_game_df = user_idx_df

In [77]:
user_owned_games_df.head(100)

,steamid,game_count,games
0,76561197960434624,1047,"[{'playtime_linux_forever': 0, 'playtime_forev..."
1,76561198015158496,[0],[]
2,76561197972495328,[0],[]
3,76561198028573552,[0],[]
4,76561197970323424,[0],[]
...,...,...,...
13037,76561198950450384,[0],[]
13038,76561198974830112,[0],[]
13039,76561198985708688,[0],[]
13040,76561198993673408,[0],[]


## (2) make data frame of user and playtime of each game

In [ ]:
import time
start = time.time()
for idx, line in enumerate(open(user_owned_games, 'r')):
    if idx % 100 == 0:
        print("time elapsed : ", time.time() - start)
    temp = json.loads(line)
    temp_data = pd.json_normalize(temp["games"], max_level = 1)
    temp_idx = user_idx_df[user_idx_df.user_id==user_owned_games_df.steamid[idx]].index.values
#     print(user_owned_games_df.steamid[idx])
#     if (temp_data[""])
#     print(user_owned_games_df.steamid[idx])
    if(temp_idx.size == 0 or temp_data.size == 0):
        continue
    if(user_owned_games_df.game_count[idx] > 1000):
        continue ;
    for app_id, play_forever in zip(temp_data["appid"], temp_data["playtime_forever"]):
        if play_forever==0:
            continue
        if not app_id in user_game_df.columns:
            user_game_df[app_id] = 0
        user_game_df.loc[temp_idx, app_id] = play_forever
#    user_game_df.
#     user_game_df.insert(temp_idx[0],
#                     temp_data.appid,
#                     temp_data.playtime_forever)

time elapsed :  0.005989789962768555
time elapsed :  292.9777331352234
time elapsed :  967.260692358017


In [ ]:
user_game_df.to_csv("user_game.csv", mode='w')

## (3) Use collaborate filletering to make recommender  system